In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
from selenium import webdriver
import time

In [4]:
SCROLL_PAUSE_TIME = 0.5

In [5]:
# obtain disease ID
diseases = pd.read_csv("gold_standard/diseases/diseases.tsv", sep = "\t")
disease_id = diseases["disease_id"][0]
print("Disease ID: " + disease_id)

Disease ID: HIF2016_DIS00


In [6]:
filename = disease_id + ".tsv"

# obtain conversations for the disease ID
conversations = pd.read_csv("gold_standard/conversations/" + filename, sep = "\t")
conversation_id = conversations["conversation_id"][0]
conversation_url = conversations["url"][0]
print("Conversation ID: " + conversation_id)
print("Conversation URL: " + conversation_url)

Conversation ID: HIF2016_DIS00_CONV00
Conversation URL: http://www.medhelp.org/posts/Allergies---Food/Does-anyone-have-an-allergy-to-coffee/show/1550820


In [8]:
# TODO: Need to scrape reviews from oldest to newest and disable multi-page: perhaps use Selenium here?
#       Extract posts based on the number needed in the dataframe

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("headless")

driver = webdriver.Chrome(options = chrome_options)
driver.get(conversation_url + "?sort_by=oldest")

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source, "html5lib")
current_post = soup.find("div", {"id": "post_show_" + 
                                 conversation_url[conversation_url.rfind("/") + 1:]})

# driver.find_element_by_id or 
# driver.find_element_by_class or
# driver.find_element_by_css or
# driver.find_element_by_xpath

FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?

In [ ]:
str(current_post)

In [ ]:
pattern = r"<div class=\"resp_body\">(.*?)</div>"
divs = re.findall(pattern, str(current_post))

In [ ]:
divs